In [1]:
import xarray as xr
from xhistogram.xarray import histogram
import numpy as np

In [5]:
ds = xr.open_dataset('../data/raw/testdata_Baltic/19000101.ocean_z_heat_1900_01.nc')
grid = xr.open_dataset('../data/raw/testdata_Baltic/19000101.ocean_static.nc')

In [36]:
def testingcalc_G(ds,lname,Ldotname,lbins,weightfactor):
    """ A very simple calculation of WMT using xhistogram.
    Intended as the standard against which to test generalized formulations.
    
    Parameters
    ----------
    ds : xr.Dataset
        Dataset containing the scalar with name lname, and the tendency with name Ldotname.
    lname : str
        Name of scalar for which to calculate WMT
    Ldotname : str
        Diagnostic name of process for which WMT will be calculated.
    lbins : np.array
        Bin edges for histogram.
    weightfactor : float or xr.DataArray
        Factor by which to multiply Ldot to convert to units of [scalar units] s-1.
        E.g. for l=temperature [units=degC], and Ldot=vertical diffusion [units : Wm-2],
        weightfactor would be [grid cell area / Cp] where Cp is the specific heat capacity.
    Returns
    -------
    G : xr.DataArray [kg s-1]
        Watermass transformation across surfaces of lname due to process Ldotname.
    """
    ### Simplest calculation of WMT due to a single process ###
    # Calculation
    nanmask = np.isnan(ds[Ldotname])
    G = histogram(ds[lname].where(~nanmask).squeeze(),
                  bins=[lbins],
                  dim=['xh','yh','z_l'],
                  weights=(weightfactor).where(~nanmask).squeeze()
                 )/np.diff(lbins)
    return G


In [37]:
# Constants
Cp=3992.
# Variables
lname = "temp"
lbins = np.linspace(-3,10,20)
Ldotname = "opottempdiff"
# Calculation
G = testingcalc_G(ds=ds,lname=lname,Ldotname=Ldotname,lbins=lbins,weightfactor=grid["areacello"]*Cp)

In [40]:
### This is a dummy function at the moment, but what I presume something like this could look like.
def test_calc_G():
    dstest = xr.open_dataset('../data/raw/testdata_Baltic/19000101.ocean_z_heat_1900_01.nc')
    gridtest = xr.open_dataset('../data/raw/testdata_Baltic/19000101.ocean_static.nc')
    
    # Simple calculation
    # Constants
    Cp=3992.
    # Variables
    lname = "temp"
    lbins = np.linspace(-3,10,20)
    Ldotname = "opottempdiff"
    # Calculation
    Gsimple = testingcalc_G(ds=ds,lname=lname,Ldotname=Ldotname,lbins=lbins,weightfactor=grid["areacello"]*Cp)
    
    # Within xwmt package framework
    G = xwmt.wmt(dstest,gridtest).G(lname,Ldotname,lbins)